# Equity & Sleeve Attribution Viewer
Plots portfolio equity, per-symbol equities, sleeve attribution, and monthly returns.


In [ ]:
import pandas as pd, plotly.graph_objects as go
from pathlib import Path
EQ_PATH = Path('data/pnl_demo_equity.csv')
TR_PATH = Path('data/pnl_demo_trades.csv')
ATTR_PATH = Path('data/pnl_demo_attrib_sleeve.csv')
eq = pd.read_csv(EQ_PATH, parse_dates=['ts']).set_index('ts')
eq.head()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=eq.index, y=eq['portfolio_equity'], name='Portfolio Equity'))
for c in eq.columns:
    if c.startswith('equity_'):
        fig.add_trace(go.Scatter(x=eq.index, y=eq[c], name=c))
fig.update_layout(title='Equity Curves', xaxis_title='UTC Date', yaxis_title='Equity (base=1.0)')
fig.show()


In [ ]:
from pathlib import Path
if Path('data/pnl_demo_attrib_sleeve.csv').exists():
    attr = pd.read_csv('data/pnl_demo_attrib_sleeve.csv', parse_dates=['exit_time'])
    daily = attr.groupby(['exit_time','sleeve'])['pnl'].sum().unstack('sleeve').fillna(0)
    daily = daily.sort_index()
    fig2 = go.Figure()
    for col in daily.columns:
        fig2.add_trace(go.Scatter(x=daily.index, y=daily[col].cumsum(), mode='lines', name=col, stackgroup='one'))
    fig2.update_layout(title='Cumulative Sleeve Attribution (stacked)')
    fig2.show()
else:
    print('No attribution file found (data/pnl_demo_attrib_sleeve.csv)')


In [ ]:
port = eq[['portfolio_equity']].copy()
port['ret'] = port['portfolio_equity'].pct_change().fillna(0)
mret = (1+port['ret']).resample('M').prod()-1
import plotly.graph_objects as go
fig3 = go.Figure()
fig3.add_bar(x=mret.index, y=mret.values, name='Monthly Return')
fig3.update_layout(title='Monthly Returns (Portfolio)', xaxis_title='Month', yaxis_title='Return')
fig3.show()
